In [376]:
%pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [173]:
from pyspark.sql import SparkSession
import os
import findspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import ast
import pandas as pd
import numpy as np
from pyspark.sql.functions import col, when, split, explode, regexp_replace, lower, lit, udf, trim

In [174]:
df_data_file = pd.read_csv("data/features_summation.csv", encoding='utf-8')

In [175]:
df_data_file.head(5)

,file,organizations,classifications,affiliations,auth-keywords,subjects,authors,reference,title,publish_name
0,201800000.json,['Department of Preventive and Social Medicine...,"['ASJC', 'SUBJABBR']","['Stanford University School of Medicine', 'Ch...",[None],['Medicine (all)'],"['Pongpirul K.', 'Lungren M.P.']","['Science.', 'The future of public health', 'I...",Public health and international epidemiology f...,"Radiology in Global Health: Strategies, Implem..."
1,201800001.json,"['Department of Electrical Engineering', 'Wire...","['ASJC', 'CPXCLASS', 'FLXCLASS', 'SUBJABBR']",['Chulalongkorn University'],[None],"['Electrical and Electronic Engineering', 'Ele...","['Pratumsiri T.', 'Janpugdee P.']","['Proc. CAMA 2015', 'Proc. 2015 Thailand-Japan...",Flexible Printed Active Antenna for Digital Te...,Progress in Electromagnetics Research Symposium
2,201800002.json,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...","['Chemistry (all)', 'Chemical Engineering (all...","['Phuakpunk K.', 'Chalermsinsuwan B.', 'Putivi...","['AICHE J.', 'Int. J. Hydrog. Energy', 'Chem. ...",Parametric study of hydrogen production via so...,Chemical Engineering Science
3,201800003.json,"['Department of Chemistry', 'Faculty of Scienc...","['CPXCLASS', 'FLXCLASS', 'ASJC', 'SUBJABBR']","['Hirosaki University', 'Chulalongkorn Univers...","['Encapsulation', 'Fluoroalkylsilane', 'Natura...","['Chemistry (all)', 'Condensed Matter Physics'...","['Saengkaew J.', 'Le D.', 'Samart C.', 'Sawada...","['Desalination', 'J. Membr. Sci.', 'Appl. Cata...",Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science
4,201800004.json,"['Program in Petrochemistry', 'Faculty of Scie...","['EMCLASS', 'ASJC', 'SUBJABBR']","['Chulalongkorn University', 'Thailand Nationa...","['acpcPNA', 'Electrochemical impedance spectro...","['Analytical Chemistry', 'Biochemistry', 'Envi...","['Teengam P.', 'Siangproh W.', 'Tuantranont A....","['Int. J. Tubercul. Lung Dis.', 'Lancet Infect...",Electrochemical impedance-based DNA sensor usi...,Analytica Chimica Acta


## Drop column

In [176]:
df_data_file.columns

Index(['file', 'organizations', 'classifications', 'affiliations',
       'auth-keywords', 'subjects', 'authors', 'reference', 'title',
       'publish_name'],
      dtype='object')

In [177]:
df_data_file.drop(columns=['file', 'authors', 'reference'], inplace=True)

In [178]:
df_data_file.describe(include='all')

,organizations,classifications,affiliations,auth-keywords,subjects,title,publish_name
count,20216,20216,20216,20216,20216,20215,20216
unique,17209,38,10528,16304,3105,20140,5501
top,['Chulalongkorn University'],"['ASJC', 'SUBJABBR']",['Chulalongkorn University'],[None],['Multidisciplinary'],Preface,Scientific Reports
freq,180,7769,4225,3762,1049,18,438


## Remove null

In [179]:
df_data_file_null_out = df_data_file

In [180]:
df_data_file_null_out = df_data_file_null_out.applymap(lambda x: None if x == '[None]' or x == '[]' or x == 'None' else x)

C:\Users\Yiwahpsp\AppData\Local\Temp\ipykernel_24164\4193919111.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_data_file_null_out = df_data_file_null_out.applymap(lambda x: None if x == '[None]' or x == '[]' or x == 'None' else x)


In [181]:
df_data_file_null_out.isnull().sum()

organizations        22
classifications       0
affiliations          0
auth-keywords      3762
subjects              0
title                 1
publish_name          0
dtype: int64

In [182]:
df_data_file_null_out.dropna(inplace=True)

In [183]:
df_data_file_null_out.drop_duplicates(inplace=True)

In [184]:
df_data_file_null_out.describe(include='all')

,organizations,classifications,affiliations,auth-keywords,subjects,title,publish_name
count,16440,16440,16440,16440,16440,16440,16440
unique,14099,38,8591,16290,2822,16428,4685
top,['Chulalongkorn University'],"['ASJC', 'SUBJABBR']",['Chulalongkorn University'],"['Beyond Standard Model', 'Hadron-Hadron scatt...",['Medicine (all)'],Axial load-strain relationships of partially e...,Journal of High Energy Physics
freq,127,6066,3537,18,531,2,231


In [185]:
df_data_file_null_out.to_csv('data_pandas/1_features_drop_null.csv', index=False, encoding='utf-8')

In [242]:
df_data_file = df_data_file_null_out

In [243]:
df_data_file_null_out.shape

(16440, 7)

In [244]:
df_data_file.shape

(16440, 7)

## Subject

In [310]:
df_data_file_subject = pd.read_csv('data_pandas/1_features_drop_null.csv', encoding='utf-8')

In [296]:
df_data_file_subject = df_data_file

In [298]:
df_data_file_subject.shape

(16440, 7)

In [311]:
df_data_file_subject.describe(include='all')

,organizations,classifications,affiliations,auth-keywords,subjects,title,publish_name
count,16440,16440,16440,16440,16440,16440,16440
unique,14099,38,8591,16290,2822,16428,4685
top,['Chulalongkorn University'],"['ASJC', 'SUBJABBR']",['Chulalongkorn University'],"['Beyond Standard Model', 'Hadron-Hadron scatt...",['Medicine (all)'],Axial load-strain relationships of partially e...,Journal of High Energy Physics
freq,127,6066,3537,18,531,2,231


In [312]:
# Ensure the subjects column contains valid string representations of lists
def safe_literal_eval(val):
	try:
		return ast.literal_eval(val)
	except (ValueError, SyntaxError):
		return []

# Split the subjects column into multiple rows
df_data_file_subject['subjects'] = df_data_file_subject['subjects'].apply(safe_literal_eval)

In [313]:
df_data_file_subject = df_data_file_subject.explode('subjects').reset_index(drop=True)

In [314]:
df_data_file_subject.shape

(42863, 7)

In [315]:
df_data_file_subject.head(5)

,organizations,classifications,affiliations,auth-keywords,subjects,title,publish_name
0,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Chemistry (all),Parametric study of hydrogen production via so...,Chemical Engineering Science
1,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Chemical Engineering (all),Parametric study of hydrogen production via so...,Chemical Engineering Science
2,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Industrial and Manufacturing Engineering,Parametric study of hydrogen production via so...,Chemical Engineering Science
3,"['Department of Chemistry', 'Faculty of Scienc...","['CPXCLASS', 'FLXCLASS', 'ASJC', 'SUBJABBR']","['Hirosaki University', 'Chulalongkorn Univers...","['Encapsulation', 'Fluoroalkylsilane', 'Natura...",Chemistry (all),Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science
4,"['Department of Chemistry', 'Faculty of Scienc...","['CPXCLASS', 'FLXCLASS', 'ASJC', 'SUBJABBR']","['Hirosaki University', 'Chulalongkorn Univers...","['Encapsulation', 'Fluoroalkylsilane', 'Natura...",Condensed Matter Physics,Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science


In [316]:
df_data_file_subject.shape

(42863, 7)

In [317]:
# Clean subject values
df_data_file_subject['subject'] = (
    df_data_file_subject['subjects']
    .str.replace(r'[\[\]\"\']', '', regex=True)  # Remove unwanted characters
    .str.strip()                                # Trim whitespace
    .str.replace(r'\(.*?\)', '', regex=True)    # Remove parentheses and their contents
    .str.lower()                                # Convert to lowercase
)

def is_valid_utf8(text):
    try:
        text.encode('utf-8')
        return True
    except UnicodeEncodeError:
        return False

# Filter rows with only valid UTF-8 characters
df_data_file_subject['subject'] = df_data_file_subject['subject'].apply(lambda x: x if is_valid_utf8(x) else np.nan)

# Drop the original subjects column (if required)
df_data_file_subject.drop(columns=['subjects'], inplace=True)

In [322]:
df_data_file_subject.isnull().sum()

organizations      0
classifications    0
affiliations       0
auth-keywords      0
title              0
publish_name       0
subject            0
dtype: int64

In [323]:
df_data_file_subject.dropna(inplace=True)

In [324]:
df_data_file_subject.head(5)

,organizations,classifications,affiliations,auth-keywords,title,publish_name,subject
0,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Parametric study of hydrogen production via so...,Chemical Engineering Science,chemistry
1,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Parametric study of hydrogen production via so...,Chemical Engineering Science,chemical engineering
2,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Parametric study of hydrogen production via so...,Chemical Engineering Science,industrial and manufacturing engineering
3,"['Department of Chemistry', 'Faculty of Scienc...","['CPXCLASS', 'FLXCLASS', 'ASJC', 'SUBJABBR']","['Hirosaki University', 'Chulalongkorn Univers...","['Encapsulation', 'Fluoroalkylsilane', 'Natura...",Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,chemistry
4,"['Department of Chemistry', 'Faculty of Scienc...","['CPXCLASS', 'FLXCLASS', 'ASJC', 'SUBJABBR']","['Hirosaki University', 'Chulalongkorn Univers...","['Encapsulation', 'Fluoroalkylsilane', 'Natura...",Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,condensed matter physics


In [325]:
df_data_file_subject.shape

(42850, 7)

In [326]:
df_data_file_subject.to_csv('data_pandas/2_features_subject.csv', index=False, encoding='utf-8')

In [327]:
df_data_file = df_data_file_subject

## Mapping Subject

In [389]:
import json
with open('clustering_subject.json', encoding='utf-8') as file:
  mapping_subjects = json.load(file)

In [390]:
mapping_subjects[:1]

[{'Cluster': 'Physics and Astronomy',
  'Subjects': ['physics',
   'astronomy',
   'astrophysics',
   'space and planetary science',
   'nuclear physics',
   'high energy physics',
   'statistical and nonlinear physics',
   'mathematical physics',
   'condensed matter physics']}]

In [398]:
df_data_file_subject_clustering = pd.read_csv('data_pandas/2_features_subject.csv', encoding='utf-8')

In [392]:
df_data_file_subject_clustering.describe()

,organizations,classifications,affiliations,auth-keywords,title,publish_name,subject
count,42850,42850,42850,42850,42850,42850,42850
unique,14099,38,8591,16290,16428,4685,293
top,['Chulalongkorn University'],"['ASJC', 'SUBJABBR']",['Chulalongkorn University'],"['Beyond Standard Model', 'Hadron-Hadron scatt...",Techno-economic sensitivity analysis for optim...,Sustainability (Switzerland),materials science
freq,309,13853,9684,18,12,734,807


In [399]:
df_data_file_subject_clustering.head(5)

,organizations,classifications,affiliations,auth-keywords,title,publish_name,subject
0,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Parametric study of hydrogen production via so...,Chemical Engineering Science,chemistry
1,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Parametric study of hydrogen production via so...,Chemical Engineering Science,chemical engineering
2,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Parametric study of hydrogen production via so...,Chemical Engineering Science,industrial and manufacturing engineering
3,"['Department of Chemistry', 'Faculty of Scienc...","['CPXCLASS', 'FLXCLASS', 'ASJC', 'SUBJABBR']","['Hirosaki University', 'Chulalongkorn Univers...","['Encapsulation', 'Fluoroalkylsilane', 'Natura...",Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,chemistry
4,"['Department of Chemistry', 'Faculty of Scienc...","['CPXCLASS', 'FLXCLASS', 'ASJC', 'SUBJABBR']","['Hirosaki University', 'Chulalongkorn Univers...","['Encapsulation', 'Fluoroalkylsilane', 'Natura...",Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,condensed matter physics


In [403]:
import nltk
from nltk.stem import PorterStemmer
from fuzzywuzzy import fuzz

subject_cluster_mapping = {cluster_data['Cluster']: set(cluster_data['Subjects']) for cluster_data in mapping_subjects}
stemmer = PorterStemmer()

def map_subjects(subject):
    subject = subject.lower()
    tokens = subject.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    for cluster, subjects in subject_cluster_mapping.items():
        for cluster_subject in subjects:
            cluster_tokens = cluster_subject.lower().split()
            stemmed_cluster_tokens = [stemmer.stem(token) for token in cluster_tokens]

            # Exact match
            if set(stemmed_tokens) == set(stemmed_cluster_tokens):
                return cluster

            # Fuzzy matching based on token similarity
            fuzzy_ratio = fuzz.token_sort_ratio(subject, cluster_subject)
            if fuzzy_ratio > 80:
                return cluster

    return 'Other'

df_data_file_subject_clustering['subject-cluster'] = df_data_file_subject_clustering['subject'].apply(lambda x: map_subjects(x.strip()))

In [404]:
df_data_file_subject_clustering.head(5)

,organizations,classifications,affiliations,auth-keywords,title,publish_name,subject,subject-cluster
0,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Parametric study of hydrogen production via so...,Chemical Engineering Science,chemistry,Chemistry
1,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Parametric study of hydrogen production via so...,Chemical Engineering Science,chemical engineering,Engineering
2,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",Parametric study of hydrogen production via so...,Chemical Engineering Science,industrial and manufacturing engineering,Engineering
3,"['Department of Chemistry', 'Faculty of Scienc...","['CPXCLASS', 'FLXCLASS', 'ASJC', 'SUBJABBR']","['Hirosaki University', 'Chulalongkorn Univers...","['Encapsulation', 'Fluoroalkylsilane', 'Natura...",Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,chemistry,Chemistry
4,"['Department of Chemistry', 'Faculty of Scienc...","['CPXCLASS', 'FLXCLASS', 'ASJC', 'SUBJABBR']","['Hirosaki University', 'Chulalongkorn Univers...","['Encapsulation', 'Fluoroalkylsilane', 'Natura...",Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,condensed matter physics,Physics and Astronomy


In [420]:
df_data_file_subject_clustering['subject-cluster'].value_counts()

subject-cluster
Physics and Astronomy                          10840
Medicine                                        5387
Biology                                         4119
Social Sciences                                 4057
Multidisciplinary                               3960
Engineering                                     3384
Chemistry                                       3270
Other                                           2482
Computer Science and Information Technology     2470
Environmental Science                           1970
Agriculture and Food Science                     911
Name: count, dtype: int64

In [419]:
def map_subjects(subject):
    subject = subject.lower()
    tokens = subject.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    for cluster, subjects in subject_cluster_mapping.items():
        for cluster_subject in subjects:
            if subject == cluster_subject.lower():
                return cluster

    return 'Other'

# Filter subjects that were previously classified as 'Other'
other_subjects = df_data_file_subject_clustering[df_data_file_subject_clustering['subject-cluster'] == 'Other']['subject']

# Re-classify 'Other' subjects
other_subjects['new_cluster'] = other_subjects.apply(map_subjects)

# Update the original DataFrame
df_data_file_subject_clustering.loc[df_data_file_subject_clustering['subject-cluster'] == 'Other', 'subject-cluster'] = other_subjects['new_cluster']

C:\Users\Yiwahpsp\AppData\Local\Temp\ipykernel_24164\2242654720.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_subjects['new_cluster'] = other_subjects.apply(map_subjects)


In [ ]:
df_data_file_subject_clustering.drop(columns=['subject'], inplace=True)

In [424]:
df_data_file_subject_clustering.describe()

,organizations,classifications,affiliations,auth-keywords,title,publish_name,subject-cluster
count,42850,42850,42850,42850,42850,42850,42850
unique,14099,38,8591,16290,16428,4685,11
top,['Chulalongkorn University'],"['ASJC', 'SUBJABBR']",['Chulalongkorn University'],"['Beyond Standard Model', 'Hadron-Hadron scatt...",Techno-economic sensitivity analysis for optim...,Sustainability (Switzerland),Physics and Astronomy
freq,309,13853,9684,18,12,734,10840


In [423]:
df_data_file_subject_clustering.drop(columns=['subject'], inplace=True)

In [425]:
df_data_file_subject_clustering.to_csv('data_pandas/2.5_features_subject_cluster.csv', index=False, encoding='utf-8')

## Publish name and Title

In [432]:
df_data_file_pb_name_title = pd.read_csv('data_pandas/2.5_features_subject_cluster.csv', encoding='utf-8')

In [427]:
df_data_file_pb_name_title = df_data_file

In [433]:
df_data_file_pb_name_title.shape

(42850, 7)

In [434]:
# Clean the 'publish_name' column
def is_valid_utf8(text):
    try:
        text.encode('utf-8')
        return True
    except UnicodeEncodeError:
        return False

df_data_file_pb_name_title['publish_name'] = df_data_file_pb_name_title['publish_name'].str.replace(r'[\[\]\"\']', '', regex=True).str.strip().str.lower()
df_data_file_pb_name_title['title'] = df_data_file_pb_name_title['title'].str.replace(r'[\[\]\"\']', '', regex=True).str.strip().str.lower()

# Filter rows with invalid UTF-8 encoding
df_data_file_pb_name_title = df_data_file_pb_name_title[df_data_file_pb_name_title['publish_name'].apply(is_valid_utf8) & df_data_file_pb_name_title['title'].apply(is_valid_utf8)]

# Additional cleaning (optional)
df_data_file_pb_name_title['title'] = df_data_file_pb_name_title['title'].str.replace('[^\w\s]', '')  # Remove non-alphanumeric characters


<>:16: SyntaxWarning: invalid escape sequence '\w'
<>:16: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Yiwahpsp\AppData\Local\Temp\ipykernel_24164\565023138.py:16: SyntaxWarning: invalid escape sequence '\w'
  df_data_file_pb_name_title['title'] = df_data_file_pb_name_title['title'].str.replace('[^\w\s]', '')  # Remove non-alphanumeric characters


In [436]:
df_data_file_pb_name_title.head(5)

,organizations,classifications,affiliations,auth-keywords,title,publish_name,subject-cluster
0,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
1,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Engineering
2,['Center of Excellence in Catalysis and Cataly...,"['CPXCLASS', 'ENCOMPASSCLASS', 'FLXCLASS', 'AS...",['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Engineering
3,"['Department of Chemistry', 'Faculty of Scienc...","['CPXCLASS', 'FLXCLASS', 'ASJC', 'SUBJABBR']","['Hirosaki University', 'Chulalongkorn Univers...","['Encapsulation', 'Fluoroalkylsilane', 'Natura...",superhydrophobic coating from fluoroalkylsilan...,applied surface science,Chemistry
4,"['Department of Chemistry', 'Faculty of Scienc...","['CPXCLASS', 'FLXCLASS', 'ASJC', 'SUBJABBR']","['Hirosaki University', 'Chulalongkorn Univers...","['Encapsulation', 'Fluoroalkylsilane', 'Natura...",superhydrophobic coating from fluoroalkylsilan...,applied surface science,Physics and Astronomy


In [435]:
df_data_file_pb_name_title.shape

(42850, 7)

In [437]:
df_data_file_pb_name_title.isnull().sum()

organizations      0
classifications    0
affiliations       0
auth-keywords      0
title              0
publish_name       0
subject-cluster    0
dtype: int64

In [438]:
df_data_file_pb_name_title.to_csv('data_pandas/3_features_publish_name_and_title.csv', index=False, encoding='utf-8')

In [439]:
df_data_file = df_data_file_pb_name_title

## Classification

In [440]:
df_data_file_classification = pd.read_csv('data_pandas/3_features_publish_name_and_title.csv', encoding='utf-8')

In [210]:
df_data_file_classification = df_data_file

In [441]:
df_data_file_classification.describe(include='all')

,organizations,classifications,affiliations,auth-keywords,title,publish_name,subject-cluster
count,42850,42850,42850,42850,42850,42850,42850
unique,14099,38,8591,16290,16421,4671,11
top,['Chulalongkorn University'],"['ASJC', 'SUBJABBR']",['Chulalongkorn University'],"['Beyond Standard Model', 'Hadron-Hadron scatt...",q-meter: quality monitoring system for telecom...,sustainability (switzerland),Physics and Astronomy
freq,309,13853,9684,18,12,734,10840


In [442]:
df_data_file_classification['classifications'] = df_data_file_classification['classifications'].apply(ast.literal_eval)

In [443]:
df_data_file_classification = df_data_file_classification.explode('classifications').reset_index(drop=True)

In [444]:
df_data_file_classification.drop_duplicates

<bound method DataFrame.drop_duplicates of                                             organizations classifications  \
0       ['Center of Excellence in Catalysis and Cataly...        CPXCLASS   
1       ['Center of Excellence in Catalysis and Cataly...  ENCOMPASSCLASS   
2       ['Center of Excellence in Catalysis and Cataly...        FLXCLASS   
3       ['Center of Excellence in Catalysis and Cataly...            ASJC   
4       ['Center of Excellence in Catalysis and Cataly...        SUBJABBR   
...                                                   ...             ...   
138609  ['Faculty of Applied Science and Center for Pr...            ASJC   
138610  ['Faculty of Applied Science and Center for Pr...        SUBJABBR   
138611  ['Faculty of Applied Science and Center for Pr...        GEOCLASS   
138612  ['Faculty of Applied Science and Center for Pr...            ASJC   
138613  ['Faculty of Applied Science and Center for Pr...        SUBJABBR   

                                

In [445]:
df_data_file_classification.shape

(138614, 7)

In [446]:
df_data_file_classification.head(5)

,organizations,classifications,affiliations,auth-keywords,title,publish_name,subject-cluster
0,['Center of Excellence in Catalysis and Cataly...,CPXCLASS,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
1,['Center of Excellence in Catalysis and Cataly...,ENCOMPASSCLASS,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
2,['Center of Excellence in Catalysis and Cataly...,FLXCLASS,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
3,['Center of Excellence in Catalysis and Cataly...,ASJC,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
4,['Center of Excellence in Catalysis and Cataly...,SUBJABBR,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry


In [447]:
human_readable_mapping = {
    'ASJC': 'all subject journal classification',
    'SUBJABBR': 'subject abbreviation',
    'EMCLASS': 'emerging class',
    'FLXCLASS': 'flexible classification',
    'CPXCLASS': 'complex classification',
    'CABSCLASS': 'cabinet classification',
    'GEOCLASS': 'geographic classification',
    'ENCOMPASSCLASS': 'encompass classification'
}

# Replace abbreviations with human-readable terms
df_data_file_classification['classifications'] = df_data_file_classification['classifications'].replace(human_readable_mapping)

In [448]:
df_data_file_classification['classifications'].value_counts()

classifications
all subject journal classification    42850
subject abbreviation                  42850
flexible classification               15506
complex classification                15369
emerging class                        12623
cabinet classification                 4078
geographic classification              3852
encompass classification               1486
Name: count, dtype: int64

In [449]:
df_data_file_classification = df_data_file_classification.rename(columns={'classifications': 'classification'})

In [450]:
df_data_file_classification.columns

Index(['organizations', 'classification', 'affiliations', 'auth-keywords',
       'title', 'publish_name', 'subject-cluster'],
      dtype='object')

In [451]:
df_data_file_classification.shape

(138614, 7)

In [452]:
df_data_file_classification.drop_duplicates(inplace=True)

In [453]:
df_data_file_classification.shape

(103242, 7)

In [454]:
df_data_file_classification.head(5)

,organizations,classification,affiliations,auth-keywords,title,publish_name,subject-cluster
0,['Center of Excellence in Catalysis and Cataly...,complex classification,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
1,['Center of Excellence in Catalysis and Cataly...,encompass classification,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
2,['Center of Excellence in Catalysis and Cataly...,flexible classification,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
3,['Center of Excellence in Catalysis and Cataly...,all subject journal classification,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
4,['Center of Excellence in Catalysis and Cataly...,subject abbreviation,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry


In [455]:
df_data_file_classification.to_csv('data_pandas/4_features_classification.csv', index=False, encoding='utf-8')

In [456]:
df_data_file = df_data_file_classification

## Organization

In [496]:
df_data_file_organization = pd.read_csv('data_pandas/4_features_classification.csv', encoding='utf-8')

In [458]:
df_data_file_classification.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103242 entries, 0 to 138610
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   organizations    103242 non-null  object
 1   classification   103242 non-null  object
 2   affiliations     103242 non-null  object
 3   auth-keywords    103242 non-null  object
 4   title            103242 non-null  object
 5   publish_name     103242 non-null  object
 6   subject-cluster  103242 non-null  object
dtypes: object(7)
memory usage: 6.3+ MB


In [463]:
df_data_file_organization.describe()

,organizations,classification,affiliations,auth-keywords,title,publish_name,subject-cluster
count,103242,103242,103242,103242,103242,103242,103242
unique,14099,8,8591,16290,16421,4671,11
top,"['Department of Computer Engineering', 'Facult...",all subject journal classification,['Chulalongkorn University'],"['feast/famine feeding', 'glycerol', 'mixed cu...",the potential of biodiesel production from gra...,acm international conference proceeding series,Physics and Astronomy
freq,640,32135,22522,36,36,1578,24342


In [497]:
def remove_university(org_list):
  if org_list is None:
    return []
  return [org for org in org_list if org and 'university' not in org.lower()]

df_data_file_organization['organizations'] = df_data_file_organization['organizations'].apply(ast.literal_eval).apply(remove_university)
df_data_file_organization.head()

,organizations,classification,affiliations,auth-keywords,title,publish_name,subject-cluster
0,[Center of Excellence in Catalysis and Catalyt...,complex classification,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
1,[Center of Excellence in Catalysis and Catalyt...,encompass classification,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
2,[Center of Excellence in Catalysis and Catalyt...,flexible classification,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
3,[Center of Excellence in Catalysis and Catalyt...,all subject journal classification,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry
4,[Center of Excellence in Catalysis and Catalyt...,subject abbreviation,['Chulalongkorn University'],"['Circulating fluidized bed', 'Computational f...",parametric study of hydrogen production via so...,chemical engineering science,Chemistry


In [471]:
df_col_organizations = df_data_file_organization['organizations']

In [475]:
df_col_organizations.head()

0    Center of Excellence in Catalysis and Catalyti...
0                   Department of Chemical Engineering
0                               Faculty of Engineering
0                                Fuels Research Center
0                    Department of Chemical Technology
Name: organizations, dtype: object

In [478]:
df_col_organizations.shape

(27844,)

In [474]:
df_col_organizations =  df_col_organizations.explode()

In [477]:
df_col_organizations.drop_duplicates(inplace=True)

In [483]:
df_data_file_organization.describe()

,organizations,classification,affiliations,auth-keywords,title,publish_name,subject-cluster
count,103242,103242,103242,103242,103242,103242,103242
unique,13768,8,8591,16290,16421,4671,11
top,[],all subject journal classification,['Chulalongkorn University'],"['feast/famine feeding', 'glycerol', 'mixed cu...",the potential of biodiesel production from gra...,acm international conference proceeding series,Physics and Astronomy
freq,966,32135,22522,36,36,1578,24342


In [485]:
df_data_file_organization['organizations'].value_counts()

organizations
[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         966
[Department of Computer Engineering]                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [498]:
df_data_file_organization = df_data_file_organization[df_data_file_organization['organizations'].apply(lambda x: x != [] and x is not None)]

In [507]:
df_data_file_organization.describe()

,organizations,classification,affiliations,auth-keywords,title,publish_name,subject-cluster
count,102276,102276,102276,102276,102276,102276,102276
unique,13767,8,8555,16051,16183,4588,11
top,['Department of Computer Engineering'],all subject journal classification,['Chulalongkorn University'],"['Beyond Standard Model', 'Hadron-Hadron scatt...",polyhydroxyalkanoate production using two-stag...,acm international conference proceeding series,Physics and Astronomy
freq,918,31745,21910,36,36,1518,24133


In [506]:
df_data_file_organization['organizations'] = df_data_file_organization['organizations'].apply(
    lambda x: str(x) if isinstance(x, list) else str(x)
)

In [508]:
df_data_file_organization.drop_duplicates(inplace=True)

In [509]:
df_data_file_organization.dropna(inplace=True)

In [510]:
df_data_file_organization.shape

(102276, 7)

In [511]:
df_data_file_organization.to_csv('data_pandas/5_features_organization.csv', index=False, encoding='utf-8')

In [512]:
df_data_file = df_data_file_organization

## Author keyword

In [513]:
df_data_file_author_key = pd.read_csv('data_pandas/5_features_organization.csv', encoding='utf-8')

In [514]:
df_data_file_author_key.describe()

,organizations,classification,affiliations,auth-keywords,title,publish_name,subject-cluster
count,102276,102276,102276,102276,102276,102276,102276
unique,13767,8,8555,16051,16183,4588,11
top,['Department of Computer Engineering'],all subject journal classification,['Chulalongkorn University'],"['Beyond Standard Model', 'Hadron-Hadron scatt...",polyhydroxyalkanoate production using two-stag...,acm international conference proceeding series,Physics and Astronomy
freq,918,31745,21910,36,36,1518,24133


In [517]:
df_data_file_author_key['auth-keywords'] = df_data_file_author_key['auth-keywords'].apply(ast.literal_eval)

In [521]:
df_data_file_author_key = df_data_file_author_key.reset_index(drop=True)
df_data_file_author_key = df_data_file_author_key.explode('auth-keywords')

In [523]:
df_data_file_author_key.head(5)

,organizations,classification,affiliations,auth-keywords,title,publish_name,subject-cluster
0,['Center of Excellence in Catalysis and Cataly...,complex classification,['Chulalongkorn University'],Circulating fluidized bed,parametric study of hydrogen production via so...,chemical engineering science,Chemistry
0,['Center of Excellence in Catalysis and Cataly...,complex classification,['Chulalongkorn University'],Computational fluid dynamics,parametric study of hydrogen production via so...,chemical engineering science,Chemistry
0,['Center of Excellence in Catalysis and Cataly...,complex classification,['Chulalongkorn University'],Multiphase flow models,parametric study of hydrogen production via so...,chemical engineering science,Chemistry
0,['Center of Excellence in Catalysis and Cataly...,complex classification,['Chulalongkorn University'],Riser,parametric study of hydrogen production via so...,chemical engineering science,Chemistry
0,['Center of Excellence in Catalysis and Cataly...,complex classification,['Chulalongkorn University'],Sorption enhanced steam methane reforming,parametric study of hydrogen production via so...,chemical engineering science,Chemistry


In [522]:
df_data_file_author_key.shape

(519480, 7)

In [538]:
# Clean subject values
df_data_file_subject['auth-keywords'] = (
    df_data_file_subject['auth-keywords']
    .str.replace(r'[\[\]\"\']', '', regex=True)  # Remove unwanted characters
    .str.strip()                                # Trim whitespace
    .str.lower()                                # Convert to lowercase
)

def is_valid_utf8(text):
    try:
        text.encode('utf-8')
        return True
    except UnicodeEncodeError:
        return False

# Filter rows with only valid UTF-8 characters
df_data_file_subject['auth-keywords'] = df_data_file_subject['auth-keywords'].apply(lambda x: x if is_valid_utf8(x) else np.nan)


In [526]:
df_data_file_author_key.shape

(519480, 7)

In [539]:
df_data_file_author_key.head(5)

,organizations,classification,affiliations,auth-keywords,publish_name,subject-cluster
0,['Center of Excellence in Catalysis and Cataly...,complex classification,['Chulalongkorn University'],Circulating fluidized bed,chemical engineering science,Chemistry
0,['Center of Excellence in Catalysis and Cataly...,complex classification,['Chulalongkorn University'],Computational fluid dynamics,chemical engineering science,Chemistry
0,['Center of Excellence in Catalysis and Cataly...,complex classification,['Chulalongkorn University'],Multiphase flow models,chemical engineering science,Chemistry
0,['Center of Excellence in Catalysis and Cataly...,complex classification,['Chulalongkorn University'],Riser,chemical engineering science,Chemistry
0,['Center of Excellence in Catalysis and Cataly...,complex classification,['Chulalongkorn University'],Sorption enhanced steam methane reforming,chemical engineering science,Chemistry


In [534]:
df_data_file_author_key.dropna(inplace=True)

In [535]:
df_data_file_author_key.drop_duplicates(inplace=True)

In [536]:
df_data_file_author_key.shape

(516673, 6)

In [532]:
df_data_file_author_key.drop(columns=['title'], inplace=True)

In [541]:
df_data_file_author_key.to_csv('data_pandas/6_features_author_keywords.csv', index=False, encoding='utf-8')

## outlier auth keyword

In [548]:
df_data_file_cutoff = pd.read_csv('data_pandas/6_features_author_keywords.csv', encoding='utf-8')

In [549]:
import numpy as np

# Apply -log transformation to all columns
df_transformed = df_data_file_cutoff.applymap(lambda x: -np.log(x) if np.issubdtype(type(x), np.number) and x > 0 else x)

# Display the transformed DataFrame
print(df_transformed.head())

C:\Users\Yiwahpsp\AppData\Local\Temp\ipykernel_24164\17225381.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_transformed = df_data_file_cutoff.applymap(lambda x: -np.log(x) if np.issubdtype(type(x), np.number) and x > 0 else x)


                                       organizations          classification  \
0  ['Center of Excellence in Catalysis and Cataly...  complex classification   
1  ['Center of Excellence in Catalysis and Cataly...  complex classification   
2  ['Center of Excellence in Catalysis and Cataly...  complex classification   
3  ['Center of Excellence in Catalysis and Cataly...  complex classification   
4  ['Center of Excellence in Catalysis and Cataly...  complex classification   

                   affiliations                              auth-keywords  \
0  ['Chulalongkorn University']                  Circulating fluidized bed   
1  ['Chulalongkorn University']               Computational fluid dynamics   
2  ['Chulalongkorn University']                     Multiphase flow models   
3  ['Chulalongkorn University']                                      Riser   
4  ['Chulalongkorn University']  Sorption enhanced steam methane reforming   

                   publish_name subject-cluster  


In [551]:
df_transformed.dropna(inplace=True)

In [552]:
df_transformed.drop_duplicates(inplace=True)

In [553]:
df_transformed.describe()

,organizations,classification,affiliations,auth-keywords,publish_name,subject-cluster
count,516669,516669,516669,516669,516669,516669
unique,13767,8,8555,50955,4588,11
top,['Department of Computer Engineering'],all subject journal classification,['Chulalongkorn University'],Thailand,international journal of molecular sciences,Physics and Astronomy
freq,4353,160099,106170,3542,7785,120436


In [555]:
df_data_file_cutoff = df_transformed

In [556]:
# Calculate the 1st and 99th percentiles
lower_percentile = df_data_file_cutoff['auth-keywords'].value_counts().quantile(0.01)
upper_percentile = df_data_file_cutoff['auth-keywords'].value_counts().quantile(0.99)

# Filter the DataFrame to keep only the rows with 'auth-keywords' within the desired range
df_data_file_cutoff = df_data_file_cutoff[
  df_data_file_cutoff['auth-keywords'].map(df_data_file_cutoff['auth-keywords'].value_counts()).between(lower_percentile, upper_percentile)
]

# Display the shape of the new DataFrame
df_data_file_cutoff.shape

(443017, 6)

In [557]:
# Calculate the 1st and 99th percentiles
lower_percentile = df_data_file_cutoff['affiliations'].value_counts().quantile(0.01)
upper_percentile = df_data_file_cutoff['affiliations'].value_counts().quantile(0.99)

# Filter the DataFrame to keep only the rows with 'auth-keywords' within the desired range
df_data_file_cutoff = df_data_file_cutoff[
  df_data_file_cutoff['affiliations'].map(df_data_file_cutoff['affiliations'].value_counts()).between(lower_percentile, upper_percentile)
]

# Display the shape of the new DataFrame
df_data_file_cutoff.shape

(281359, 6)

In [558]:
# Calculate the 1st and 99th percentiles
lower_percentile = df_data_file_cutoff['publish_name'].value_counts().quantile(0.01)
upper_percentile = df_data_file_cutoff['publish_name'].value_counts().quantile(0.99)

# Filter the DataFrame to keep only the rows with 'auth-keywords' within the desired range
df_data_file_cutoff = df_data_file_cutoff[
  df_data_file_cutoff['publish_name'].map(df_data_file_cutoff['publish_name'].value_counts()).between(lower_percentile, upper_percentile)
]

# Display the shape of the new DataFrame
df_data_file_cutoff.shape

(226080, 6)

In [559]:
# Calculate the 1st and 99th percentiles
lower_percentile = df_data_file_cutoff['organizations'].value_counts().quantile(0.01)
upper_percentile = df_data_file_cutoff['organizations'].value_counts().quantile(0.99)

# Filter the DataFrame to keep only the rows with 'auth-keywords' within the desired range
df_data_file_cutoff = df_data_file_cutoff[
  df_data_file_cutoff['organizations'].map(df_data_file_cutoff['organizations'].value_counts()).between(lower_percentile, upper_percentile)
]

# Display the shape of the new DataFrame
df_data_file_cutoff.shape

(215953, 6)

In [560]:
df_data_file_cutoff_temp = df_data_file_cutoff

In [561]:
# Calculate the 1st and 99th percentiles
lower_percentile = df_data_file_cutoff['classification'].value_counts().quantile(0.01)
upper_percentile = df_data_file_cutoff['classification'].value_counts().quantile(0.99)

# Filter the DataFrame to keep only the rows with 'auth-keywords' within the desired range
df_data_file_cutoff = df_data_file_cutoff[
  df_data_file_cutoff['classification'].map(df_data_file_cutoff['classification'].value_counts()).between(lower_percentile, upper_percentile)
]

# Display the shape of the new DataFrame
df_data_file_cutoff.shape

(214663, 6)

In [563]:
df_data_file_cutoff.dropna(inplace=True)

C:\Users\Yiwahpsp\AppData\Local\Temp\ipykernel_24164\285248224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_file_cutoff.dropna(inplace=True)


In [564]:
df_data_file_cutoff.drop_duplicates(inplace=True)

C:\Users\Yiwahpsp\AppData\Local\Temp\ipykernel_24164\232068504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_file_cutoff.drop_duplicates(inplace=True)


In [565]:
df_data_file_cutoff.describe()

,organizations,classification,affiliations,auth-keywords,publish_name,subject-cluster
count,214663,214663,214663,214663,214663,214663
unique,8632,7,7464,29898,3384,11
top,"['Department of Anatomic Pathology', 'Departme...",all subject journal classification,"['Kasetsart University, Kamphaeng Saen Campus'...",atrial fibrillation,aids,Physics and Astronomy
freq,100,70726,254,72,747,51973


In [566]:
df_data_file_cutoff.to_csv('data_pandas/7_classification_cutoff.csv', index=False, encoding='utf-8')

In [567]:
df_data_file_cutoff.head(10)

,organizations,classification,affiliations,auth-keywords,publish_name,subject-cluster
51,"['Department of Chemistry', 'Faculty of Scienc...",complex classification,"['Hirosaki University', 'Chulalongkorn Univers...",Fluoroalkylsilane,applied surface science,Chemistry
53,"['Department of Chemistry', 'Faculty of Scienc...",complex classification,"['Hirosaki University', 'Chulalongkorn Univers...",Oil/water separation superhydrophobic/superole...,applied surface science,Chemistry
55,"['Department of Chemistry', 'Faculty of Scienc...",flexible classification,"['Hirosaki University', 'Chulalongkorn Univers...",Fluoroalkylsilane,applied surface science,Chemistry
57,"['Department of Chemistry', 'Faculty of Scienc...",flexible classification,"['Hirosaki University', 'Chulalongkorn Univers...",Oil/water separation superhydrophobic/superole...,applied surface science,Chemistry
59,"['Department of Chemistry', 'Faculty of Scienc...",all subject journal classification,"['Hirosaki University', 'Chulalongkorn Univers...",Fluoroalkylsilane,applied surface science,Chemistry
61,"['Department of Chemistry', 'Faculty of Scienc...",all subject journal classification,"['Hirosaki University', 'Chulalongkorn Univers...",Oil/water separation superhydrophobic/superole...,applied surface science,Chemistry
63,"['Department of Chemistry', 'Faculty of Scienc...",subject abbreviation,"['Hirosaki University', 'Chulalongkorn Univers...",Fluoroalkylsilane,applied surface science,Chemistry
65,"['Department of Chemistry', 'Faculty of Scienc...",subject abbreviation,"['Hirosaki University', 'Chulalongkorn Univers...",Oil/water separation superhydrophobic/superole...,applied surface science,Chemistry
67,"['Department of Chemistry', 'Faculty of Scienc...",complex classification,"['Hirosaki University', 'Chulalongkorn Univers...",Fluoroalkylsilane,applied surface science,Physics and Astronomy
69,"['Department of Chemistry', 'Faculty of Scienc...",complex classification,"['Hirosaki University', 'Chulalongkorn Univers...",Oil/water separation superhydrophobic/superole...,applied surface science,Physics and Astronomy
